In [1]:
import torch

import data
import utils
import info_recorder as ir
import data_loader as dl
import initializer as init
import trainer as tn
import tester as ts

In [2]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


There are 1 GPU(s) available.
Device name: NVIDIA GeForce GTX 1070


In [3]:
df, size_info, size_dict = data.get_df()

In [4]:
sourceCode_np = df.sourceCode.values
codeClass_np = df.classLabel.values

In [5]:
# Tokenize, build vocabulary, encode tokens
print("Tokenizing...\n")
tokenized_sourceCodes, ch2idx, max_len = utils.tokenize(sourceCode_np)
input_ids = utils.encode(tokenized_sourceCodes, ch2idx, max_len)

Tokenizing...



In [6]:
ir.record_ch2idx(ch2idx)

In [7]:
encoded_class2idx, class2idx, num_classes = utils.tokenize_encode_class(codeClass_np)

In [8]:
from sklearn.model_selection import train_test_split

train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    input_ids, encoded_class2idx, test_size = 0.1, random_state = 43
)

In [9]:
train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    train_inputs, train_labels, test_size = 0.1, random_state = 43
)

In [10]:
# Load data to PyTorch DataLoader
train_dataloader, val_dataloader, test_dataloader = dl.data_loader(train_inputs, val_inputs, test_inputs, train_labels, val_labels, test_labels, batch_size=50)

In [11]:
# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('dataDivided/tests')

In [12]:
# CNN-rand: Word vectors are randomly initialized.
tn.set_seed(42)
cnn_rand, optimizer = init.initilize_model(device=device,
                                           vocab_size=len(ch2idx),
                                           embed_dim=100,
                                           hidden_size=100,
                                           num_classes=len(class2idx),
                                           n_layers=3,
                                           dropout=0.2,
                                           learning_rate=0.25,
                                           optimizerName="Adadelta",
                                           modelType="RNN")
                                        
tn.train(device, cnn_rand, optimizer, train_dataloader, 'DataDiv_test1', writer, val_dataloader, epochs=20)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------
   1    |   2.835396   |  2.507455  |   22.12   |  186.83  
   2    |   2.364505   |  2.204464  |   29.66   |  188.84  
   3    |   1.967934   |  1.751515  |   46.56   |  189.07  
   4    |   1.581424   |  1.438681  |   56.63   |  189.05  
   5    |   1.195723   |  1.018788  |   70.12   |  189.06  
   6    |   0.939195   |  0.846983  |   75.52   |  189.08  
   7    |   0.802293   |  0.753734  |   78.33   |  189.09  
   8    |   0.715465   |  0.703265  |   79.75   |  189.13  
   9    |   0.651707   |  0.660692  |   81.00   |  189.12  
  10    |   0.603531   |  0.616633  |   82.31   |  189.12  
  11    |   0.564858   |  0.608738  |   82.34   |  189.07  
  12    |   0.534426   |  0.579580  |   83.47   |  189.08  
  13    |   0.506800   |  0.563590  |   84.04   |  189.06  
  14    |   0.482717   |  0.553264  |   84.21   |  189.12  
  15    |   0.461143

In [ ]:
ts.test(device, cnn_rand, test_dataloader)

test loss:  0.5851330412463063
test acc:  83.62857142857143
